Este notebook recorre los archivos html de la carpeta `.data`, y exporta de ahi el js del cuatrimestre
Ese `horarios.js` hay que ponerlo en la carpeta src/data
La carpeta data tiene que tener un archivo html por **cada carrera**
El html se consigue de la siguiente manera
- Loggearse al SIU, ir a "Reportes" > "Oferta de comisiones"
- Conseguir el HTML de ese sitio (CTRL+U)
- Ese HTML abrirlo en un editor de texto y buscar el texto `kernel.renderer.on_arrival`
- Esa funcion recibe un objeto. Ese objeto tiene una clave `content`
- El **valor** de content es el html que va en la carpeta `.data`

In [1]:
import os

for f in os.listdir():
    if '.html' not in f:
        continue
    
    with open(f, 'r+') as fsub:
        txt = fsub.read()
        txt = txt.replace(r'\"', r'"')
        txt = txt.replace(r'<\/', r'</')
        txt = txt.replace(r'\n', '')
        txt = txt.replace(r'\t', '')
        txt = txt.encode('iso-8859-1').decode('unicode_escape')
        fsub.seek(0)
        fsub.write(txt)
        fsub.truncate()

/home/delmazo/.local/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: invalid escape sequence '\/'
  del sys.path[0]


In [2]:
from bs4 import BeautifulSoup
from collections import namedtuple
import re
import json
import codecs

REGEX = r'(.*) (\(\d\d\d\d\))'
materia_regex = re.compile(REGEX)
DIAS_DE_LA_HERMOSA_SEMANA = ["Domingo", "Lunes", "Martes", "Miercoles", "Jueves", "Viernes", "Sabado"]

In [3]:
def get_ids_materias(soup):
    codigos_materias = []
    actividades = soup.find_all('div', {'class': 'js-recuadro_actividad'})

    for act in actividades:
        titulo, codigo = materia_regex.search(act.get('actividad')).groups()
        codigo = re.sub('\(|\)', "", codigo)
        codigos_materias.append(codigo)
    return codigos_materias

soup = BeautifulSoup(open("sistemas.html"), "html.parser")
get_ids_materias(soup)

['9142',
 '9524',
 '8102',
 '9514',
 '9515',
 '9502',
 '9135',
 '8101',
 '9504',
 '9564',
 '9505',
 '8636',
 '9568',
 '9559',
 '9141',
 '9530',
 '9139',
 '9535',
 '9802',
 '9803',
 '9801',
 '9804',
 '9805',
 '9525',
 '9140',
 '9548',
 '9124',
 '8111',
 '9520',
 '9521',
 '9104',
 '9105',
 '9106',
 '9558',
 '9557',
 '8104',
 '9560',
 '9519',
 '9503',
 '9508',
 '9509',
 '9506',
 '9507']

In [4]:
def get_ids_comisiones(soup):
    ids_comisiones = []
    comisiones_soup = soup.find_all('table', {'class': 'comision'})
    for comision in comisiones_soup:
        comision_id = comision.get('comision')
        ids_comisiones.append(comision_id)
    return ids_comisiones

def get_materias(soup):
    materias = []
    actividades = soup.find_all('div', {'class': 'js-recuadro_actividad'})

    for act in actividades:
        titulo, codigo = materia_regex.search(act.get('actividad')).groups()
        codigo = re.sub('\(|\)', "", codigo)
        ids_comisiones = get_ids_comisiones(act)
        
        materias.append({
            "codigo": codigo,
            "titulo": titulo,
            "comisiones": ids_comisiones
        })
        
    return materias

soup = BeautifulSoup(open("informatica.html"), "html.parser")
get_materias(soup)

[{'codigo': '7544',
  'titulo': 'ADM. Y CONTROL DE PROY. INFORMATICOS I',
  'comisiones': ['29543']},
 {'codigo': '7546',
  'titulo': 'ADM. Y CONTROL DE PROY. INFORMATICOS II',
  'comisiones': ['29545']},
 {'codigo': '6108',
  'titulo': 'ALGEBRA II A',
  'comisiones': ['29579',
   '29581',
   '29582',
   '29583',
   '29584',
   '29585',
   '29586',
   '29587',
   '29588',
   '29589',
   '29590',
   '29591',
   '29592',
   '29593',
   '29594',
   '29595',
   '29596',
   '29597',
   '29598',
   '31529',
   '31530',
   '31531']},
 {'codigo': '7540',
  'titulo': 'ALGORITMOS Y PROGRAMACION I',
  'comisiones': ['29606', '29608', '29609', '29610']},
 {'codigo': '7541',
  'titulo': 'ALGORITMOS Y PROGRAMACION II',
  'comisiones': ['29623', '29624', '29625', '29626']},
 {'codigo': '7507',
  'titulo': 'ALGORITMOS Y PROGRAMACION III',
  'comisiones': ['29635', '29636', '31646']},
 {'codigo': '6606',
  'titulo': 'ANALISIS DE CIRCUITOS',
  'comisiones': ['31551', '31556', '31564']},
 {'codigo': '750

In [5]:
def get_comision(soup, codigo):
    comision_soup = soup.find('table', {'comision': codigo})
    docentes = comision_soup.get('docentes')
    comision_id = comision_soup.get('comision')
    clases = []

    for dia_semana_soup in comision_soup.find_all('tr', {'class': ['js-dia']}):
        dia_semana = dia_semana_soup.get('dia_sem')
        horas_soup = dia_semana_soup.find('td', text=re.compile(r'\d\d\:\d\d'))
        if not horas_soup:
            continue
        horas = horas_soup.contents[0]
        inicio, fin = horas.split(' a ')

        clases.append({'dia': DIAS_DE_LA_HERMOSA_SEMANA.index(dia_semana), 'inicio': inicio, 'fin': fin})

    return {
        "docentes": docentes,
        "codigo": codigo,
        "clases": clases
    }

soup = BeautifulSoup(open("informatica.html"), "html.parser")
get_comision(soup, "31348")

{'docentes': 'FERRIGNO LEANDRO SEBASTIÁN',
 'codigo': '31348',
 'clases': [{'dia': 1, 'inicio': '18:00', 'fin': '22:00'}]}

In [6]:
carreras = []
codigos_materias_agregadas = []
materias = []
codigos_comisiones_agregadas = []
comisiones = []

for f in os.listdir():
    if '.html' not in f:
        continue
        
    soup = BeautifulSoup(open(f), "html.parser")

    carreras.append({
        "nombre": f.split('.')[0],
        'materias': get_ids_materias(soup)
    })
    
    materias_carrera = get_materias(soup)
    materias_carrera = [x for x in materias_carrera if x['codigo'] not in codigos_materias_agregadas]
    for m in materias_carrera:
        codigos_materias_agregadas.append(m['codigo'])
        comisiones_materia = m.get('comisiones')
        comisiones_materia = [x for x in comisiones_materia if x not in codigos_comisiones_agregadas]
        for c in comisiones_materia:
            codigos_comisiones_agregadas.append(c)
            comisiones.append(get_comision(soup, c))
    
    materias.extend(materias_carrera)
    
    
    
obj = {
    "carreras": carreras,
    "materias": materias,
    "comisiones": comisiones
}

obj_dump = json.dumps(obj, indent=2, ensure_ascii=False)        

with codecs.open('horarios.js', 'w', encoding='utf-8') as fw:
    fw.write("export const data = ")
    fw.write(obj_dump)

print(obj_dump)    

{
  "carreras": [
    {
      "nombre": "informatica",
      "materias": [
        "7544",
        "7546",
        "6108",
        "7540",
        "7541",
        "7507",
        "6606",
        "7509",
        "6103",
        "6110",
        "7512",
        "7538",
        "7141",
        "7573",
        "6626",
        "7515",
        "7548",
        "6619",
        "6669",
        "6405",
        "7112",
        "6670",
        "7118",
        "7558",
        "6201",
        "6203",
        "6215",
        "7802",
        "7803",
        "7801",
        "7804",
        "7805",
        "7113",
        "7146",
        "7543",
        "7550",
        "6602",
        "6609",
        "7140",
        "7514",
        "7566",
        "6107",
        "7201",
        "6211",
        "7114",
        "7115",
        "7120",
        "6620",
        "7506",
        "7556",
        "6109",
        "6647",
        "6646",
        "6675",
        "6301",
        "7144",
        "6663",
        "6632